# Feature Pipeline


Run the full data pipeline and inspect engineered features.

In [1]:
# Environment setup
import sys, subprocess
from pathlib import Path

print('Python:', sys.executable)
repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
subprocess.run(['pip', 'install', '-e', str(repo_root)], check=True)

Python: /Users/pratik_n/Downloads/gridpulse/.venv/bin/python
Obtaining file:///Users/pratik_n/Downloads/gridpulse
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for gridpulse (pyproject.toml): started
  Building editable for gridpulse (pyproject.toml): finished with status 'done'
  Created wheel for gridpulse: filename=gridpulse-0.1.0-0.editable-py3-none-any.whl size=6240 sha256=a3db1f09c05cc91ada05789e7fdc9ba2f25cfdf999b001c94d7cef295a71a594
  Stored in directory: /private/var/folders/pk/ksqc41pj60q3589gfwhqz5tw0000gn/T/pip-

CompletedProcess(args=['pip', 'install', '-e', '/Users/pratik_n/Downloads/gridpulse'], returncode=0)

In [2]:
from pathlib import Path
import shutil

repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
raw_csv = repo_root / 'data' / 'raw' / 'time_series_60min_singleindex.csv'
if not raw_csv.exists():
    nested = next((repo_root / 'data' / 'raw').glob('opsd-time_series-*/time_series_60min_singleindex.csv'), None)
    if nested:
        raw_csv.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy(nested, raw_csv)
        print('Copied:', nested, '->', raw_csv)
    else:
        print('Missing OPSD CSV. Please download or place it in data/raw/')
print('Raw CSV:', raw_csv)


Raw CSV: /Users/pratik_n/Downloads/gridpulse/data/raw/time_series_60min_singleindex.csv


In [3]:
import subprocess

subprocess.run([
    'python', '-m', 'gridpulse.data_pipeline.validate_schema',
    '--in', 'data/raw', '--report', 'reports/data_quality_report.md'
], check=True, cwd=str(repo_root))

subprocess.run([
    'python', '-m', 'gridpulse.data_pipeline.build_features',
    '--in', 'data/raw', '--out', 'data/processed'
], check=True, cwd=str(repo_root))

subprocess.run([
    'python', '-m', 'gridpulse.data_pipeline.split_time_series',
    '--in', 'data/processed/features.parquet',
    '--out', 'data/processed/splits'
], check=True, cwd=str(repo_root))


Wrote report: reports/data_quality_report.md
Saved: data/processed/features.parquet
Rows: 49481 | Columns: 39
Wrote splits to: data/processed/splits


CompletedProcess(args=['python', '-m', 'gridpulse.data_pipeline.split_time_series', '--in', 'data/processed/features.parquet', '--out', 'data/processed/splits'], returncode=0)

In [5]:
from pathlib import Path
repo_root = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
features_path = repo_root / "data" / "processed" / "features.parquet"
print(features_path, features_path.exists())


/Users/pratik_n/Downloads/gridpulse/data/processed/features.parquet True
